In [1]:
import sys
import warnings
sys.path.append('../')
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 1
%aimport utils, GroupingAlgorithm, HEEM_VQE_Functions, qiskit

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.auto import tqdm

from qiskit import IBMQ, Aer, QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.utils.quantum_instance import QuantumInstance
from qiskit.ignis.verification import get_ghz_simple
from qiskit.compiler import transpile
from qiskit.tools.monitor import job_monitor
from qiskit.opflow.state_fns import CircuitStateFn
from qiskit.providers.aer.extensions.snapshot_statevector import *

from utils import get_backend_connectivity, molecules, Label2Chain, from_string_to_numbers, load_grouping_data, current_time, load_data_IBMQ, send_ibmq_parallel
from GroupingAlgorithm import groupingWithOrder, TPBgrouping
from HEEM_VQE_Functions import measure_circuit_factor, probability2expected_binary, post_process_results, probability2expected_parallel

In [62]:
def compute_energy(Groups, Measurements, method, n_chunks=10, layout=None):
    
    circuits = [measure_circuit_factor(measurement, n_qubits, measure_all=False).compose(state_0, front=True) for measurement
                 in Measurements]
    
    kwards_run = {'coupling_map': coupling_map, 'noise_model': noise_model, 'basis_gates': basis_gates, 'shots': NUM_SHOTS // len(Groups)}
    
    if layout is not None:
        kwards_run['initial_layout'] = layout[::-1]
    
    jobs = send_ibmq_parallel(backed_calculations, n_chunks, circuits, job_tag=[molecule_name, method], verbose=False,
                              progress_bar=True, output_file='progress_' + method + '.txt', kwards_run=kwards_run)

    counts = []
    
    for job in jobs:
        counts += job.result().get_counts()
    
    energy = 0
    pbar = tqdm(range(len(counts)), desc='Computing energy')
    for j in pbar:
        probabilities = post_process_results(counts[j], circuits[j].num_clbits, kwards_run['shots'])
        diagonals, factors = probability2expected_binary(coeffs, labels, [Groups[j]], [Measurements[j]], shift=False)
        diagonals = [(~diagonal * 2 - 1).astype('int8') for diagonal in diagonals[0]]

        energy += np.sum((diagonals * np.array(factors[0])[:, None]) @ probabilities)

    return jobs, energy

In [5]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    IBMQ.load_account()

provider_main = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider_CSIC = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic')

name_backend = 'ibmq_montreal'
backend = provider_CSIC.get_backend(name_backend)
WC_device = get_backend_connectivity(backend)

G_device = nx.Graph()
G_device.add_edges_from(WC_device)

In [6]:
NUM_SHOTS = 2 ** 14

backend_hpc = provider_main.get_backend('ibmq_qasm_simulator')
backed_simulation = provider_main.get_backend('simulator_statevector')
simulator = Aer.get_backend('aer_simulator')  # Backend for simulation

device = QasmSimulator.from_backend(backend)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates

backed_calculations = backend_hpc

In [78]:
molecule_name = 'C2H6'

try:
    qubit_op = np.load('../data/big_molecules.npy', allow_pickle=True).item()[molecule_name]
    print('Data loaded')
except KeyError:
    print('Computing molecule')
    qubit_op = molecules(molecule_name)
        
paulis, coeffs, labels = Label2Chain(qubit_op)

n_qubits = qubit_op.num_qubits
state_0 = QuantumCircuit(n_qubits)
# state_0 = state_0.compose(get_ghz_simple(n_qubits, measure=False))  # Initialized in the GHZ state

print(f'{len(paulis)} total Pauli strings')
print(f'{n_qubits} qubits')

Data loaded
8919 total Pauli strings
26 qubits


# Exact analytical

## $|0\rangle^{\otimes N}$

In [79]:
energy_exact = 0

for i in range(len(labels)):
    label = labels[i]
    if 'X' not in label and 'Y' not in label:
        energy_exact += coeffs[i]
print('Exact energy: {}'.format(energy_exact))

Exact energy: -11.419431103002799


## $|\mathrm{GHZ}\rangle$

In [70]:
diagonals = {'I': np.array([1, 0, 0, 1]), 'X': np.array([0, 1, 1, 0]),
             'Y': np.array([0, 1j, -1j, 0]), 'Z': np.array([1, 0, 0, -1])}

energy_exact = 0
for label, coeff in zip(labels, coeffs):
    diagonal = np.ones(4, dtype=complex)
    for op in label:
        diagonal *= diagonals[op]
    energy_exact += coeff * np.sum(diagonal) / 2
    
print('Exact energy: {}'.format(energy_exact.real))

Exact energy: -13.647455702351461


# Exact (Numerical)

psi = CircuitStateFn(state_0)
energy_exact = psi.adjoint().compose(qubit_op).compose(psi).eval().real
    
print('Exact energy: {}'.format(energy_exact))

# Exact (Simulation)

In [8]:
Groups, Measurements, _ = load_grouping_data(molecule_name, 'EM')
# Groups, Measurements, _ = groupingWithOrder(paulis, connected=True, print_progress=True)
print('Number of groups: {}'.format(len(Groups)))

circuits_exact = []
for j, measure in enumerate(Measurements):
    circuit = measure_circuit_factor(measure, n_qubits, make_measurements=False)
    circuit = circuit.compose(state_0, front=True)
    # circuit.save_statevector()
    circuit.snapshot_statevector('snapshot')
    circuits_exact.append(circuit)

Data loaded
Number of groups: 682


C:\Users\david\anaconda3\envs\heem\lib\site-packages\qiskit\providers\aer\extensions\snapshot_statevector.py:86: DeprecationWarning: The `SnapshotStatevector` instruction will be deprecated in thefuture. It has been superseded by the `SaveStatevector` instructions.
  SnapshotStatevector(label, num_qubits=len(snapshot_register)),
C:\Users\david\anaconda3\envs\heem\lib\site-packages\qiskit\providers\aer\extensions\snapshot_statevector.py:49: DeprecationWarning: The `Snapshot` instruction will be deprecated in the future. It has been superseded by the `SaveStatevector` instructions.
  super().__init__(label, snapshot_type='statevector', num_qubits=num_qubits)


In [9]:
depth = []
for circuit in circuits_exact:
    depth.append(circuit.depth())
    
print(np.average(depth), np.median(depth), np.max(depth))

3.1510263929618767 3.0 4


In [10]:
n_max = 25

if n_max == -1:
    n_max = len(circuits_exact)
n_runs = int(np.ceil(len(circuits_exact) / n_max))

In [27]:
jobs = send_ibmq_parallel(backed_calculations, n_max, circuits_exact, job_tag=molecule_name, kwards_run={'shots': 1}, progress_bar=True)
jobs_ids = [job.job_id() for job in jobs]

np.save(molecule_name + '_exact_job_ids.npy', jobs_ids)

Jobs completed:   0%|          | 0/5 [00:00<?, ?it/s]

Job 1/5 sent to IBMQ
Job 2/5 sent to IBMQ
Job 3/5 sent to IBMQ
Job 4/5 sent to IBMQ
Job 5/5 sent to IBMQ


In [11]:
jobs_ids = np.load(molecule_name + '_exact_job_ids.npy').tolist()

In [13]:
energy_exact = 0

pbar = tqdm(total=len(jobs_ids), desc='Computing exact energy')
for i in range(len(jobs_ids)):
    prob = load_data_IBMQ(jobs_ids[i], verbose=True)
    
    initial = i * n_max
    final = min(initial + n_max, len(circuits_exact))
    
    temp = probability2expected_parallel(-1, coeffs, labels, Groups[initial:final], Measurements[initial:final], shift=True, print_progress=True)
    diagonals = [x[0] for x in temp]
    factors = [x[1] for x in temp]
    
    diagonals = [(~diagonal * 2 - 1).astype('int8') for diagonal in diagonals]
    
    for j in range(len(diagonals)):
        energy_exact += np.sum((diagonals[j] * np.array(factors[j])[:, None]) @ prob[j])
        
    pbar.update()
pbar.close()
print('Exact energy: {}'.format(energy_exact))

Computing exact energy:   0%|          | 0/28 [00:00<?, ?it/s]

data downloaded
Loading data...


Computing diagonal factors:   0%|          | 0/24 [00:00<?, ?it/s]

KeyboardInterrupt: 

# TPB

In [80]:
temp = load_grouping_data(molecule_name, 'TPB')
# temp = None

if temp is not None:
    Groups, Measurements = temp
else:
    _, Groups, Measurements = TPBgrouping(paulis, print_progress=True)

print('Number of groups', len(Groups))

Data loaded
Number of groups 2069


In [ ]:
method = 'TPB'
jobs_TPB, energy_TPB = compute_energy(Groups, Measurements, method, n_chunks=100)
relative_TPB = np.abs((energy_TPB - energy_exact) / energy_exact)

print('TPB energy: {}'.format(energy_TPB))
print('Relative error: {:.3f} %'.format(relative_TPB * 100))

time = sum((job.time_per_step()['COMPLETED'] - job.time_per_step()['RUNNING']).total_seconds() for job in jobs_TPB)
print(f'Simulation time {time} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_TPB))

Jobs completed:   0%|          | 0/21 [00:00<?, ?it/s]

# EM

In [ ]:
temp = load_grouping_data(molecule_name, 'EM')
# temp = None

if temp is not None:
    Groups, Measurements, layout = temp
else:
    Groups, Measurements, layout = groupingWithOrder(paulis, connected=True, print_progress=True)

print('Number of groups', len(Groups))

In [ ]:
method = 'EM'

jobs_EM, energy_EM = compute_energy(Groups, Measurements, method, layout=layout, n_chunks=25)
relative_EM= np.abs((energy_EM - energy_exact) / energy_exact)

print('EM energy: {}'.format(energy_EM))
print('Relative error: {:.3f} %'.format(relative_EM * 100))

time = sum((job.time_per_step()['COMPLETED'] - job.time_per_step()['RUNNING']).total_seconds() for job in jobs_EM)
print(f'Simulation time {time} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_EM))

# HEEM

In [ ]:
temp = load_grouping_data(molecule_name, 'HEEM')
# temp = None

if temp is not None:
    Groups, Measurements, layout = temp
else:
    Groups, Measurements, layout = groupingWithOrder(paulis, G_device, connected=True, print_progress=True)

print('Number of groups', len(Groups))

In [ ]:
method = 'HEEM'

jobs_HEEM, energy_HEEM = compute_energy(Groups, Measurements, method, n_chunks=50, layout=layout)
relative_HEEM = np.abs((energy_HEEM - energy_exact) / energy_exact)

print('HEEM energy: {}'.format(energy_HEEM))
print('Relative error: {:.3f} %'.format(relative_HEEM * 100))

time = sum((job.time_per_step()['COMPLETED'] - job.time_per_step()['RUNNING']).total_seconds() for job in jobs_HEEM)
print(f'Simulation time {time} s')

with open('progress_' + method + '.txt', 'w') as f:
    f.write('Done: {}\n'.format(energy_HEEM))